<center><img src="https://imgcdn.stablediffusionweb.com/2025/1/3/355bcb46-cdbe-4632-9e49-ba0b79fd662e.jpg"/></center>

In [ ]:
import chess
import chess.svg
from stockfish import Stockfish
import re
import random

In [ ]:
from IPython.display import SVG, display, clear_output

In [ ]:
from ollama import chat
from ollama import ChatResponse

In [ ]:
def pgn_of_moves(move_list):
    s = ""
    for i, (a,b) in enumerate(move_list):
        s += f"{i+1}.{a} {b} "
    return s

def moves_of_pgn(opening):
    # Split the opening string by spaces
    tokens = opening.split()
    
    # Initialize an empty list to store moves
    list_moves = []

    # Iterate through tokens, skipping the numeric labels
    i = 0
    while i < len(tokens):
        if tokens[i].endswith('.'):  # Skip move numbers (e.g., '1.', '2.')
            i += 1
        else:
            # Remove numeric prefix if present and add pairs of moves (white and black)
            white_move = tokens[i].split('.')[-1]
            black_move = tokens[i + 1] if i + 1 < len(tokens) and not tokens[i + 1].endswith('.') else None
            list_moves.append((white_move, black_move))
            i += 2 if black_move else 1

    return list_moves

In [ ]:
opening='1.e4 e5 2.Nf3 Nc6 3.Bb5 a6 4.Ba4 Nf6'

In [ ]:
moves_of_pgn(opening)

In [ ]:
pgn_of_moves(moves_of_pgn(opening))

In [ ]:
def extract_next_move(input_string):
    """
    Extracts content between <next_move> and </next_move> tags from the input string.

    Args:
        input_string (str): The string containing the tags and content to parse.

    Returns:
        str: The content between the <next_move> tags, or None if the tags are not found.
    """
    match = re.search(r"<next_move>(.*?)</next_move>", input_string, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

def parse_move(input_string):
    """
    Checks if the input string starts with an integer followed by a dot and returns the integer and the rest of the string.

    Args:
        input_string (str): The string to test and parse.

    Returns:
        tuple: A tuple containing the integer and the string after the dot, or None if the pattern does not match.
    """
    match = re.match(r"^(\d+)\.(.*)$", input_string)
    if match:
        integer_part = int(match.group(1))
        rest_of_string = match.group(2).strip()
        return integer_part, rest_of_string
    return None
    
prompt = """Here is a start of a chess game using Portable Game Notation (PGN):
{pgn_moves}
What would you play next? Explain your thought process and give your recommendation. Your recommendation should be written in PGN and contain only the last move.
Please format your answer as follows:
<explanation>[your thought process]</explanation>
<next_move>[{i}.your move in PGN]</next_move>
"""

In [ ]:
class Game:
    def __init__(self, llm, elo=1200):
        self.board = chess.Board()
        self.elo = elo
        self.stockfish = Stockfish(
        path="/opt/homebrew/bin/stockfish/",
        depth=3,
        parameters={
            "Threads": 2,
            "Minimum Thinking Time": 30,
            "UCI_Elo": self.elo,
            },
        )
        self.llm = llm
        self.not_finish = True
        self.logs = []  # Store print messages

    def log(self, message):
        """Stores a message to the log and prints it."""
        self.logs.append(message)

    def display_logs_and_board(self, size=400):
        """Displays the logs and current board in Jupyter Notebook."""
        clear_output(wait=True)  # Clear previous output
        
        # Display the board
        board_svg = chess.svg.board(self.board, size=size)
        display(SVG(board_svg))
        # Display logs
        for log in self.logs:
            print(log)

    def update_moves(self, move_w, move_b):
        for move in (move_w, move_b):
            board_move = self.board.parse_san(move)
            self.board.push(board_move)
            self.stockfish.make_moves_from_current_position([ board_move ])

    def start_game(self, opening):
        self.move_list = moves_of_pgn(opening)
        for (move_w, move_b) in self.move_list:
            self.update_moves(move_w,move_b)

    def nxt_move(self):
        nxt_move_llm = self.llm.nxt_move_llm(self.move_list)
        if nxt_move_llm is None:
            print('Parsing error')
            check = False
            self.not_finish = False
        else:
            try:
                move_san = nxt_move_llm[1]
                move_llm = self.board.parse_san(move_san)
                check = True
            except Exception as err:
                self.log(f"Illegal move {err=}, {type(err)=}")
                check = False
                self.not_finish = False # stop after first lilegal move
        
        if not check:
            move_llm = random.choice(list(self.board.legal_moves))
            move_san = self.board.san(move_llm)
        self.board.push(move_llm)
        self.stockfish.make_moves_from_current_position([ move_llm ])
        self.log(f"LLM Move: {move_san}")
        self.display_logs_and_board()  # Display the board after LLM's move
        if self.board.is_game_over():
            print("GAME-OVER")
            print(self.board.outcome())
            self.not_finish = False
        else:
            best = self.stockfish.get_best_move()
            self.stockfish.make_moves_from_current_position([ best ])
            move = chess.Move.from_uci(best)
            st_san = self.board.san(move)
            self.board.push(move)
            self.move_list.append((move_san, st_san))
            self.log(f"Stockfish Move: {st_san}")
            self.display_logs_and_board()  # Display the board after Stockfish's move
            if self.board.is_game_over():
                print("GAME-OVER")
                print(self.board.outcome())
                self.not_finish = False
        
    def play_game(self, opening):
        self.start_game(opening)
        self.display_logs_and_board()  # Display the initial logs and board
        while self.not_finish:
            self.nxt_move()
    

class LLM:
    def __init__(self, model='llama3.2', prompt=prompt):
        self.model = model
        self.prompt = prompt
        

    def make_prompt(self, move_list):
        pgn_moves = pgn_of_moves(move_list)
        prompt_chess = self.prompt.format(pgn_moves=pgn_moves, i=len(move_list)+1)
        return prompt_chess

    def nxt_move_llm(self, move_list, verbose = True):
        prompt_chess = self.make_prompt(move_list)
        response: ChatResponse = chat(model=self.model, messages=[
          {
            'role': 'user',
            'content': prompt_chess,
          },
        ])
        if verbose:
            print(response.message.content)
        next_move = extract_next_move(response.message.content)
        if next_move:
            return parse_move(next_move)
        else:
            return None
        

In [ ]:
llm = LLM('mistral')

In [ ]:
game = Game(llm)

In [ ]:
game.play_game('')

# Playing with nanogpt

![](https://github.com/karpathy/nanoGPT/raw/master/assets/nanogpt.jpg)

Following [Adam Karvonen](https://adamkarvonen.github.io/machine_learning/2024/01/03/chess-world-models.html), I trained a [nanoGPT](https://github.com/karpathy/nanoGPT/tree/master) model in order to play chess.

You can get it by following [these steps](https://github.com/dataflowr/notebooks/blob/master/llm/02_get_model.ipynb).

In [ ]:
import torch
from nanogpt.model_inf import GPTConfig, GPT
import pickle

In [ ]:
meta_path = "nanogpt/meta.pkl" # do not change if you followed the instructions in the 02_get_model

with open(meta_path, "rb") as f:
    meta = pickle.load(f)
    stoi, itos = meta["stoi"], meta["itos"]

In [ ]:
device = #torch.device('cpu') or  'cuda' or 'mps' 

checkpoint_path = # where you saved your model
checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=True)

In [ ]:
gptconf = GPTConfig(**checkpoint["model_args"])

In [ ]:
model = GPT(gptconf)

In [ ]:
state_dict = checkpoint["model"]

In [ ]:
unwanted_prefix = "_orig_mod."
for k, v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        #print(k)
        state_dict[k[len(unwanted_prefix) :]] = state_dict.pop(k)

In [ ]:
model.load_state_dict(state_dict)

In [ ]:
model.eval()
model = model.to(device)

In [ ]:
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

In [ ]:
game_start = ';1.e4 e5 '

In [ ]:
start_ids = encode(game_start)
start_ids

In [ ]:
x = torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...]
top_k = None #200  # retain only the top_k most likely tokens, clamp others to have 0 probability
max_new_tokens = 400
temperature = (
            0.01  # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
        )
with torch.no_grad():
    y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)

model_response = decode(y[0].tolist())

In [ ]:
model_response

In [ ]:
y[0]

Now define a class `GPT_chess` in order to play games with the model you downloaded.

Improve it. But, you are not allowed to use the stockfish engine inside your llm! You can check that moves are valid...